In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
drive_project_folder_path='/content/drive/MyDrive/ml-ai files arjun'
datafile_path=f'{drive_project_folder_path}/tweet_emotions.csv'
# datafile_path=f'{drive_project_folder_path}/tweets.csv'


tweets_data = pd.read_csv(datafile_path)
old_tweets_data=pd.read_csv(f'{drive_project_folder_path}/tweets.csv')

tweets_data.columns=['tweet_id','sentiment','content']

In [ ]:
tweets_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [ ]:
old_tweets_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7920 non-null   int64 
 1   label   7920 non-null   int64 
 2   tweet   7920 non-null   object
dtypes: int64(2), object(1)
memory usage: 185.8+ KB


In [ ]:
tweets_data.shape

(40000, 3)

In [ ]:
## remove trouble some rows
# tweets_data.loc[6340,:]['content'] ### the trouble causing row in tweets.csv

# tweets_data.loc[[12867,   664,  1538,  3401,  2621,  9040, 11111,  9920,  8576,
#               852,  6806,  3181,  4865,  9257,  6342, 13986,  2657,  8390,
#              7244, 13810],:]


In [ ]:
tweets_data.drop([12867,   664,  1538,  3401,  2621,  9040, 11111,  9920,  8576,
              852,  6806,  3181,  4865,  9257,  6342, 13986,  2657,  8390,
             7244, 13810],axis=0,inplace=True)

In [ ]:
num_targets=tweets_data['sentiment'].nunique()
num_targets

13

In [ ]:
tweets_data['sentiment'].value_counts()

neutral       8626
worry         8458
happiness     5205
sadness       5165
love          3841
surprise      2186
fun           1776
relief        1526
hate          1323
empty          826
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [ ]:
### data too big system crashes
tweets_data=tweets_data.iloc[1:15000,:]
tweets_data.shape

(14999, 3)

In [ ]:
tweets_data.columns

Index(['tweet_id', 'sentiment', 'content'], dtype='object')

In [ ]:
tweets_data.head(15)

,tweet_id,sentiment,content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?
10,1956969456,neutral,cant fall asleep


In [ ]:
tweets_data['content'][4]

'@dannycastillo We want to trade with someone who has Houston tickets, but no one will.'

In [ ]:
tweets_data.isna().sum()

tweet_id     0
sentiment    0
content      0
dtype: int64

In [ ]:
# tweets_data['sentiment'][1]

In [ ]:
tweets_data['sentiment'].value_counts()

worry         4665
sadness       3333
neutral       2788
surprise       813
hate           786
happiness      684
love           583
relief         336
fun            335
empty          313
enthusiasm     196
boredom        103
anger           64
Name: sentiment, dtype: int64

In [ ]:
tweets_data=tweets_data.drop('tweet_id',axis=1)
X=tweets_data.drop('sentiment',axis=1)
y=tweets_data['sentiment']

In [ ]:
### encode y
from sklearn.preprocessing import LabelEncoder
oe=LabelEncoder()

# y=y.values.reshape(-1,1)
y_np=oe.fit_transform(y)
y[:]=y_np
y

1        10
2        10
3         3
4         8
5        12
         ..
15015    12
15016    12
15017     2
15018    12
15019    12
Name: sentiment, Length: 14999, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [ ]:
X_train.shape

(11999, 1)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from bs4 import BeautifulSoup
import re
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text


#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\^^", "", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

def lemmatize_text(text):
    lem=nltk.stem.wordnet.WordNetLemmatizer()
    text= ' '.join([lem.lemmatize(word) for word in text.split()])
    return text

#removing the stopwords
from nltk.tokenize.toktok import ToktokTokenizer
def remove_stopwords(text, is_lower_case=False):
    tokenizer1=ToktokTokenizer()
    stopword_list=nltk.corpus.stopwords.words('english')
    tokens = tokenizer1.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text



In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler

class DummyTransformer(BaseEstimator,TransformerMixin):

    '''
    dummy class to inherit from to avoid typing the fit method for everything
    '''
    def __init__(self):
        return None
    def fit(self,X=None,y=None):
        return self
    def transform(self,X=None):
        return X

def do_basic_text_preprocessing(text:str):
    preprocessed_text=denoise_text(text)
    preprocessed_text=remove_special_characters(preprocessed_text)
    preprocessed_text=remove_stopwords(preprocessed_text)
    preprocessed_text=lemmatize_text(preprocessed_text)


    return preprocessed_text

class TextPreprocessor(DummyTransformer):
    def transform(self,X:pd.DataFrame):
#         preprocessed_X_np_array=X.apply(do_basic_text_preprocessing).values
        # preprocessed_X_nparr=X['content'].apply(lambda x:do_basic_text_preprocessing(x)).values
        preprocessed_X_df=X['content'].apply(lambda x:do_basic_text_preprocessing(x))
        return preprocessed_X_df

class Tokenizer(DummyTransformer):
    def transform(self, X):
        X=pd.DataFrame(X)
        # print(X.info())
        # print(X)
        X_tokenized=X['content'].apply(lambda x:simple_preprocess(x))
        # X_tokenized=X.apply(word_tokenize,axis=0) ## requires downloading punkt
        # print(X_tokenized)
        return X_tokenized


def get_nan_when_regex_does_not_match(x,pattern='^\s+$'):
    if re.search(pattern,x) == None:
        return np.nan
    else:
        return 1




from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import word_tokenize

class NonGensimTokenizer(DummyTransformer):
    def transform(self, X):
        X=pd.DataFrame(X)
        # print(X.info())
        # print(X)
        print('has NaN?   ',X.isna().sum())
        temp_df=X['content'].apply(lambda x:get_nan_when_regex_does_not_match(x,'^\s*$'))
        print('has empty string ?  ',len(temp_df)-int(temp_df.isna().sum()))
        print(X.info())
        print('this is X[content] ',X['content'])
        # X_tokenized=X['content'].apply(lambda x:ToktokTokenizer.tokenize(x))
        X_tokenized=X['content'].apply(lambda x:word_tokenize(x))
        # X_tokenized=X.apply(word_tokenize,axis=0) ## requires downloading punkt
        # print(X_tokenized)
        return X_tokenized

class SparseToDenseArr(DummyTransformer):
    def transform(self,X=None):
#         preprocessed_X_np_array=X.apply(do_basic_text_preprocessing).values
        dense_arr=X.toarray()

        return dense_arr

class MetaFeatureEngineer(DummyTransformer):
    def transform(self, X=None,y=None):
        return None

tfidf_meta_union_inst=FeatureUnion([('tfidf',TfidfVectorizer()),('metafeature',MetaFeatureEngineer())])

# tfidf_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('main',tfidf_meta_union_inst),('sparsetodense',SparseToDenseArr())])
tfidf_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('main',tfidf_meta_union_inst),('std_scaler',StandardScaler())])


# tfidf_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tfidf',TfidfVectorizer()),('sparsetodense',SparseToDenseArr())])


bow_meta_union_inst=FeatureUnion([('count',CountVectorizer()),('metafeature',MetaFeatureEngineer())])

# bow_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('main',bow_meta_union_inst),('sparsetodense',SparseToDenseArr())])
# bow_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('main',bow_meta_union_inst),('sparsetodense',SparseToDenseArr()),('std_scaler',StandardScaler())])
bow_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('count',CountVectorizer()),('sparsetodense',SparseToDenseArr()),('std_scaler',StandardScaler())])




In [ ]:
### word2vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

class CustomWord2VecTransformer(DummyTransformer):
    def __init__(self,**kwargs):
        self.input_args=kwargs
        self.count=0

    def fit(self,X,y=None):
        self.w2v_model=Word2Vec(X,**self.input_args)

        return self


    def get_mean_embedding_for_doc(self,doc_tokens):
        self.count+=1
        model=self.w2v_model
        embeddings=[]
        for tok in doc_tokens:
            if tok in model.wv.index_to_key:
                embeddings.append(model.wv.get_vector(tok))
        mean_embedding=np.mean(embeddings,axis=0)
        # if self.count>2:
        #     pass
        # else:
        #     print()
        return mean_embedding

    def transform(self, X): ## successfully troubleshooted; troubleshot transform function
        # print('before  ',X.shape)

        X_transformed=X.apply(lambda x:self.get_mean_embedding_for_doc(x))
        # print('x_transformed before .values ',X_transformed)
        # print(X_transformed.shape)
        # print('x_transformed before .tolist ; info ',X_transformed.info())

        # X_transformed=pd.DataFrame(X_transformed.tolist())
        self.nan_index=X_transformed.index[X_transformed.isna()]
        # print(self.nan_index)
        X_transformed=X_transformed.dropna()
        X_transformed=pd.DataFrame(X_transformed.values.tolist()) ## still float not iterable error
        # X_transformed=pd.DataFrame(list(X_transformed.values))
        # X_transformed=pd.DataFrame(X_transformed.values)
        # print('x_transformed after .tolist ',X_transformed)
        # # print('x_transformed after .tolist ; info ',np.info(X_transformed))
        # print('x_transformed after .tolist ; info ',X_transformed.info())
        # print('x_transformed after .tolist ; dtypes ',X_transformed.dtypes)

        # print(X_transformed)
        return X_transformed

    # def transform(self, X): ### original transform
    #     print('before  ',X.shape)

    #     X_transformed=X.apply(lambda x:self.get_mean_embedding_for_doc(x))
    #     print('x_transformed before .values ',X_transformed)
    #     print(X_transformed.shape)
    #     X_transformed=pd.DataFrame(X_transformed.tolist())
    #     # X_transformed=pd.DataFrame(X_transformed.values)
    #     print('x_transformed after .values ',X_transformed)
    #     # print(X_transformed)
    #     return X_transformed


class GoogleWord2VecTransformer(DummyTransformer):
    def __init__(self,*args,**kwargs):
        self.w2vmodel=KeyedVectors.load_word2vec_format(*args,**kwargs)

        self.input_args=kwargs


    def fit(self,X,y=None):

        return self

    def get_mean_embedding_for_doc(self,doc_tokens):
        model=self.w2vmodel
        embeddings=[]
        for tok in doc_tokens:
            if tok in model:
                embeddings.append(model[tok])
        return np.mean(embeddings,axis=0)


    # def transform(self, X): ### old
    #     X_transformed=X.apply(lambda x:self.get_mean_embedding_for_doc(x))


    #     X_transformed=pd.DataFrame(X_transformed.tolist())
    #     return X_transformed

    def transform(self, X): ## successfully troubleshooted; troubleshot transform function
        # print('before  ',X.shape)

        X_transformed=X.apply(lambda x:self.get_mean_embedding_for_doc(x))
        # print('x_transformed before .values ',X_transformed)
        # print(X_transformed.shape)
        # print('x_transformed before .tolist ; info ',X_transformed.info())

        # X_transformed=pd.DataFrame(X_transformed.tolist())
        self.nan_index=X_transformed.index[X_transformed.isna()]
        # print(self.nan_index)
        X_transformed=X_transformed.dropna()
        X_transformed=pd.DataFrame(X_transformed.values.tolist()) ## still float not iterable error
        # X_transformed=pd.DataFrame(list(X_transformed.values))
        # X_transformed=pd.DataFrame(X_transformed.values)
        # print('x_transformed after .tolist ',X_transformed)
        # # print('x_transformed after .tolist ; info ',np.info(X_transformed))
        # print('x_transformed after .tolist ; info ',X_transformed.info())
        # print('x_transformed after .tolist ; dtypes ',X_transformed.dtypes)
        return X_transformed




In [ ]:
'''
# google_meta_union_inst=FeatureUnion([
#     ('google',GoogleWord2VecTransformer("/content/drive/MyDrive/ml-ai files arjun/GoogleNews-vectors-negative300.bin",binary=True)),
#     ('metafeature',MetaFeatureEngineer())])
'''

# from google.colab import drive
# drive.mount('/content/drive')
# google_trans_inst=GoogleWord2VecTransformer(f"{drive_project_folder_path}/GoogleNews-vectors-negative300.bin",binary=True)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
google_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tokenizer',Tokenizer()),('main',google_trans_inst)])
cbow_custom_w2v_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tokenizer',Tokenizer()),('main',CustomWord2VecTransformer(vector_size = 300, window = 6 , min_count = 3, sg = 0))])
skgram_w2v_pipeline=cbow_custom_w2v_pipeline.set_params(main=CustomWord2VecTransformer(vector_size = 300, window = 6 , min_count = 3, sg = 1))

# cbow_custom_w2v_pipeline=Pipeline([('textpreprocessor',TextPreprocessor()),('tokenizer',NonGensimTokenizer()),('main',CustomWord2VecTransformer(vector_size = 300, window = 6 , min_count = 3, sg = 0))])
# skgram_w2v_pipeline=cbow_custom_w2v_pipeline.set_params(main=CustomWord2VecTransformer(vector_size = 300, window = 6 , min_count = 3, sg = 1))






In [ ]:
# prepro_pipe=Pipeline([('textpreprocessor',TextPreprocessor()),('tokenizer',Tokenizer())])

# X_trans=prepro_pipe.fit_transform(X_train)
# X_trans.shape


In [ ]:
# cus=CustomWord2VecTransformer(vector_size = 300, window = 6 , min_count = 3, sg = 0)
# x_cus=cus.fit_transform(X_trans)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.svm import SVC

result_dict={}

In [ ]:


# pipelines=[tfidf_pipeline,bow_pipeline,google_pipeline,cbow_custom_w2v_pipeline,skgram_w2v_pipeline]
# pipeline_names=['tfidf_pipeline','bow_pipeline','google_pipeline','cbow_custom_w2v_pipeline','skgram_w2v_pipeline']

pipelines=[google_pipeline,skgram_w2v_pipeline,bow_pipeline] ## testing
pipeline_names=['google_pipeline','skgram_w2v_pipeline','bow_pipeline']


# pipelines=[cbow_custom_w2v_pipeline,bow_pipeline] ## testing
# pipeline_names=['cbow_custom_w2v_pipeline','bow_pipeline']

# pipelines=[bow_pipeline] ## testing
# pipeline_names=['bow_pipeline']

count=0
for pipe in pipelines:
    pipe_name=pipeline_names[count]
    y_train=pd.DataFrame(y_train).astype('category')
    y_test=pd.DataFrame(y_test).astype('category')
    X_train_transformed=pipe.fit_transform(X_train)
    dropped_nan_index_train=pipe.named_steps['main'].nan_index.tolist()
    # X_train_transformed=X_train_transformed.drop(dropped_nan_index_train,axis=0)
    y_train_nan_dropped=y_train.drop(dropped_nan_index_train,axis=0)

    X_test_transformed=pipe.transform(X_test)
    dropped_nan_index_test=pipe.named_steps['main'].nan_index.tolist()
    y_test_nan_dropped=y_test.drop(dropped_nan_index_test,axis=0)
    X_train_to_use=X_train_transformed
    X_test_to_use=X_test_transformed
    y_train_to_use=y_train_nan_dropped
    y_test_to_use=y_test_nan_dropped
    print(pipe_name)
    # type(pipe["main"]).__name__
    # algo_insts=[GaussianNB(),DecisionTreeClassifier(),LogisticRegression()] ##
    # algo_insts=[XGBClassifier(),RandomForestClassifier(),AdaBoostClassifier()] ##
    algo_insts=[RandomForestClassifier(),SVC(kernel='linear'),XGBClassifier()] ##


    count+=1
    for algo_inst in algo_insts:
        y_train_to_use=pd.DataFrame(y_train_to_use)
        y_train_to_use=y_train_to_use.values.flatten()
        algo_inst.fit(X_train_to_use, y_train_to_use)
        y_pred=algo_inst.predict(X_test_to_use)
        accuracy=accuracy_score(y_test_to_use,y_pred)
        result_dict.update({f'{pipe_name}_{type(algo_inst).__name__}':{'accuracy':accuracy}})
        print(f'{pipe_name}_{type(algo_inst).__name__}   ',accuracy)
pd.DataFrame(result_dict).transpose()



<ipython-input-321-05424db898d3>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
<ipython-input-321-05424db898d3>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


google_pipeline
google_pipeline_RandomForestClassifier    0.3436241610738255
google_pipeline_SVC    0.34328859060402683
google_pipeline_XGBClassifier    0.34496644295302015


<ipython-input-321-05424db898d3>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
<ipython-input-321-05424db898d3>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


skgram_w2v_pipeline
skgram_w2v_pipeline_RandomForestClassifier    0.3119946091644205
skgram_w2v_pipeline_SVC    0.3244609164420485


In [ ]:
pipe.named_steps['main'].nan_index

In [ ]:
y_pred

In [ ]:
print(len(y_pred))
y_pred_df=pd.DataFrame(y_pred)
print(y_pred_df.isna().sum())
pd.DataFrame(y_pred).value_counts()


In [ ]:
print(len(y_test_to_use))
y_test_to_use.value_counts()

In [ ]:
print(y_test_to_use.isna().sum())

In [ ]:
accuracy_score(y_test_to_use.astype('category'),pd.DataFrame(y_pred).astype('category'))


In [ ]:
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

text_col=X['content']
all_words = " ".join(text_col)
all_words = word_tokenize(all_words)
dist = FreqDist(all_words)
del all_words
num_unique_word = len(dist)

### get max words per document
r_len = []

for text in X:
    word = word_tokenize(text)
    l = len(word)
    r_len.append(l)

MAX_TWEET_LEN = np.max(r_len)
del r_len
class TensorflowTokenizer(DummyTransformer):
    def transform(self,X=None):
        # text_col=X['content']
        text_col=X
        tokenizer = Tokenizer(num_words = num_unique_word)
        tokenizer.fit_on_texts(list(text_col))
        text_col= tokenizer.texts_to_sequences(text_col)
        text_col = sequence.pad_sequences(text_col, MAX_TWEET_LEN)
        return text_col

keras_pipe= Pipeline([('textpreprocessor',TextPreprocessor()),('tensorflowtokenizer',TensorflowTokenizer())])

pipe = keras_pipe
X_train_transformed=pipe.fit_transform(X_train)
X_test_transformed=pipe.transform(X_test)
X_train_to_use=X_train_transformed
X_test_to_use=X_test_transformed
y_train_to_use=y_train
y_test_to_use=y_test



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding,Flatten

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = num_unique_word, output_dim = 150, input_length = MAX_TWEET_LEN))

model.add(LSTM(128, dropout = 0.2 ))

model.add(Dense(num_targets, activation = "softmax"))
model.add(Flatten())

In [ ]:
y_train_to_use=y_train_to_use.astype(np.int).flatten()

# y_train_to_use = np.asarray(y_train_to_use).astype('float32').reshape((-1,1))
# y_train_to_use

In [ ]:
from tensorflow import keras

In [ ]:
y_train_to_use=keras.utils.to_categorical(y_train_to_use,num_targets)
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])
history1 = model.fit(X_train_to_use, y_train_to_use, epochs = 60, batch_size = 32)

In [ ]:
model.metrics_names

In [ ]:
y_test_to_use=y_test_to_use.astype(np.int).flatten()
y_test_to_use=keras.utils.to_categorical(y_test_to_use,num_targets)

eval_result=model.evaluate(X_test_to_use,y_test_to_use)
result_dict.update({f'keras':{'accuracy':eval_result}})

pd.DataFrame(result_dict).transpose()['accuracy'].max()

# pd.DataFrame(result_dict).transpose()

In [ ]:
import pickle
dbfile = open(f'{drive_project_folder_path}/pipelines.pkl', 'ab')

pickle.dump(pipelines, dbfile)
dbfile.close()


dbfile = open(f'{drive_project_folder_path}/result_dict.pkl', 'ab')

pickle.dump(result_dict, dbfile)
dbfile.close()


dbfile = open(f'{drive_project_folder_path}/history1.pkl', 'ab')

pickle.dump(history1, dbfile)
dbfile.close()


In [ ]:
history1 = model.fit(X_train_to_use, y_train_to_use, epochs = 10, batch_size = 32)

In [ ]:
eval_result=model.evaluate(X_test_to_use,y_test_to_use)
result_dict.update({f'keras':{'accuracy':eval_result}})

pd.DataFrame(result_dict).transpose()['accuracy'].max()

In [ ]:
import pickle
dbfile = open(f'{drive_project_folder_path}/pipelines.pkl', 'ab')

pickle.dump(pipelines, dbfile)
dbfile.close()


dbfile = open(f'{drive_project_folder_path}/result_dict.pkl', 'ab')

pickle.dump(result_dict, dbfile)
dbfile.close()


dbfile = open(f'{drive_project_folder_path}/history1.pkl', 'ab')

pickle.dump(history1, dbfile)
dbfile.close()


In [ ]:

# X_train_tfidf=custom_w2v_pipeline.fit_transform(X_train)
# X_test_tfidf=custom_w2v_pipeline.transform(X_test)

# X_train=TextPreprocessor().transform(X_train)

In [ ]:
# print(np.sum(X_train_tfidf >0))

In [ ]:
### pass parameters to Pipeline steps

In [ ]:



# algos=[('naive_bayes','GaussianNB'),('tree','DecisionTreeClassifier'),('linear_model','LogisticRegression')]
# algos=[('linear_model','LogisticRegression')] ### why did i even do this?




# for algo in algos:
#     ## apparently the methods of the module are attributes of the obj outputted by import_module
#     class_var=getattr(import_module(f'sklearn.{algo[0]}'),algo[1])
#     clf=class_var()
#     # clf=module.
#     clf.fit(X_train_to_use, y_train_to_use)
#     y_pred=clf.predict(X_test_to_use)
#     result_dict.update({f'{type(clf).__name__}':{'accuracy':accuracy_score(y_pred,y_test_to_use)}})
# pd.DataFrame(result_dict)

In [ ]:
#### preprocess the text to that is in the required format for word2vec



In [ ]:
# print(simple_preprocess(tweets_data['content'][1])[:30])

In [ ]:
### google accuracy 0.8118686868686869
### cbow accuracy 0.8377525252525253
